In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
from tqdm import tqdm
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial import KDTree
from joblib import Parallel, delayed
import pulp
from deap import base, creator, tools, algorithms
import folium
from folium.plugins import FastMarkerCluster
import multiprocessing
from sklearn.metrics import calinski_harabasz_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 开始处理taxi数据

In [ ]:
df_taxi = pd.read_csv('/kaggle/input/mcm-jnu-a-problem/datasets/taxi.csv')
df_taxi.head()

### 画图

In [ ]:
# 提取经纬度数据
longitude = df_taxi['longitude']
latitude = df_taxi['latitude']

# 创建散点图
plt.figure(figsize=(10, 6))
plt.scatter(longitude, latitude, c='blue', marker='o', alpha=0.5)

# 设置图表标题和标签
plt.title('Scatter Plot of Longitude and Latitude')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# 显示图表
plt.grid(True)
plt.show()

### 删除数据中异常值
由于图表中存在经度和纬度都不在北京的情况，故进行一部分的数据清洗删除
查阅网络知，北京市的经度范围是115.416827——117.508251，是纬度范围39.442078——41.058964

In [ ]:
# 定义经纬度范围
longitude_min = 115.416827
longitude_max = 117.508251
latitude_min = 39.442078
latitude_max = 41.058964

# 筛选数据
df_taxi = df_taxi[
    (df_taxi['longitude'] >= longitude_min) & (df_taxi['longitude'] <= longitude_max) &
    (df_taxi['latitude'] >= latitude_min) & (df_taxi['latitude'] <= latitude_max)
    ]

# 提取经纬度数据
longitude = df_taxi['longitude']
latitude = df_taxi['latitude']

# 创建散点图
plt.figure(figsize=(10, 6))
plt.scatter(longitude, latitude, c='blue', marker='o', alpha=0.5)

# 设置图表标题和标签
plt.title('Scatter Plot of Longitude and Latitude within Specified Range')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# 设置横纵坐标范围
#plt.xlim(longitude_min, longitude_max)
#plt.ylim(latitude_min, latitude_max)

# 显示图表
plt.grid(True)
plt.show()

In [ ]:
## 处理北京路网数据
### 读取数据

In [ ]:
# 设置列的数据类型
node_dtypes = {
    'node_id': str,
    'x_coord': float,
    'y_coord': float,
}

link_dtypes = {
    'link_id': str,
    'from_node_id': str,
    'to_node_id': str,
    'length': float,
}

poi_dtypes = {
    'poi_id': str,
    'x_coord': float,
    'y_coord': float,
    'type': str,
}

# 读取 CSV 文件
df_node = pd.read_csv("/kaggle/input/mcm-jnu-a-problem/datasets/node.csv", encoding='gbk', dtype=node_dtypes, low_memory=False)
df_link = pd.read_csv("/kaggle/input/mcm-jnu-a-problem/datasets/link.csv", encoding='gbk', dtype=link_dtypes, low_memory=False)
df_poi = pd.read_csv("/kaggle/input/mcm-jnu-a-problem/datasets/poi.csv", encoding='gbk', dtype=poi_dtypes, low_memory=False)

In [ ]:
# 查看df_node数据
df_node # 道路交叉口
df_node_droped = df_node.drop(columns=['name','zone_id','node_type','activity_type','is_boundary','poi_id','notes'])

# 输出所有列名
print("df_node columns:", df_node_droped.columns.tolist())

df_node_droped

In [ ]:
df_link # 道路信息
df_link_droped = df_link.drop(columns=['capacity'])

# 输出所有列名
print("df_link columns:", df_link_droped.columns.tolist())

df_link_droped

In [ ]:
# 获取 name 列的唯一值
unique_names = df_poi['name'].dropna().unique()

# 输出唯一值
print(unique_names)

# 将唯一值保存到文件中
with open('unique_names.txt', 'w') as f:
    for name in unique_names:
        f.write(name + '\n')

print("Unique names have been saved to 'unique_names.txt'.")

df_poi # 兴趣点信息

# 轨迹数据分析
-- 行驶轨迹聚类
-- 充电需求预测
## 行驶轨迹聚类
使用聚类算法（如 K-Means、DBSCAN）对出租车轨迹数据进行聚类，找出热点区域和常见路径
[这是一个链接(聚类算法)](https://zhuanlan.zhihu.com/p/104355127)
选择 Calinski-Harabasz Index (CH Index)方法选择最佳的k值

In [ ]:
# 抽样数据，例如采样10%的数据
sampled_data = df_taxi.sample(frac=0.1, random_state=1)
coords = sampled_data[['longitude', 'latitude']].values

# Calinski-Harabasz Index 确定 K 值
ch_scores = []
k_range = range(2, 25)  # CH Index 从 2 开始
for k in tqdm(k_range, desc="Calinski-Harabasz Index Method"):
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=10000, n_init=10, random_state=1)
    cluster_labels = kmeans.fit_predict(coords)
    ch_index = calinski_harabasz_score(coords, cluster_labels)
    ch_scores.append(ch_index)

# 绘制 Calinski-Harabasz Index 图
plt.figure(figsize=(10, 6))
plt.plot(k_range, ch_scores, marker='o')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Calinski-Harabasz Index')
plt.title('Calinski-Harabasz Index Method for Determining Optimal K')
plt.grid(True)
plt.show()

# 从 Calinski-Harabasz Index 图中选择 K 值
optimal_k = 21

# 使用最佳 K 值进行 MiniBatchKMeans 聚类
kmeans = MiniBatchKMeans(n_clusters=optimal_k, batch_size=10000, n_init=10, random_state=1)
kmeans.fit(coords)
sampled_data['cluster'] = kmeans.labels_

# 打印聚类结果
print(sampled_data['cluster'].value_counts())

# 可视化聚类结果
plt.figure(figsize=(12, 8))
colors = plt.cm.nipy_spectral(np.linspace(0, 1, optimal_k))
for k, col in zip(range(optimal_k), colors):
    class_member_mask = (sampled_data['cluster'] == k)
    xy = coords[class_member_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, markersize=6, markeredgewidth=0)

plt.title('K-Means Clustering of Taxi Trajectories')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

## 充电需求预测
### 计算每辆出租车的行驶距离
通过计算每辆出租车在相邻数据点之间的距离，估算行驶距离。使用 Haversine 公式计算地球表面两点之间的距离。

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # 将角度转换为弧度
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine 公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # 地球半径（公里）
    r = 6371
    return c * r

# 创建 datetime 列
df_taxi['datetime'] = pd.to_datetime(df_taxi['date'] + ' ' + df_taxi['time'])

# 按照 taxi_id 和 datetime 排序
df_taxi.sort_values(by=['taxi_id', 'datetime'], inplace=True)

# 计算相邻点之间的距离
df_taxi['prev_longitude'] = df_taxi.groupby('taxi_id')['longitude'].shift()
df_taxi['prev_latitude'] = df_taxi.groupby('taxi_id')['latitude'].shift()
df_taxi['distance'] = haversine(df_taxi['prev_longitude'], df_taxi['prev_latitude'], df_taxi['longitude'], df_taxi['latitude'])

# 填充空值为 0（第一个点没有前一个点的距离）
df_taxi['distance'].fillna(0, inplace=True)

df_taxi

In [ ]:
# 充电需求预测

# 假设每辆出租车的续航里程为 300 公里
range_per_charge = 300

# 计算每辆出租车的累计行驶距离
df_taxi['cumulative_distance'] = df_taxi.groupby('taxi_id')['distance'].cumsum()

# 预测充电次数
df_taxi['charge_needed'] = (df_taxi['cumulative_distance'] // range_per_charge).astype(int)

# 检查计算结果
print(df_taxi.head())

# 统计每辆出租车的充电次数
charging_stats = df_taxi.groupby('taxi_id')['charge_needed'].max().reset_index()
print(charging_stats.head())


In [ ]:
# 统计每辆出租车的充电需求
charging_stats = df_taxi.groupby('taxi_id')['charge_needed'].max().reset_index()

# 总体充电需求统计
total_charging_sessions = charging_stats['charge_needed'].sum()
print(f"总计需要的充电次数: {total_charging_sessions}")

# 可视化充电需求分布
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(charging_stats['charge_needed'], bins=20, edgecolor='k', alpha=0.7)
plt.title('Distribution of Charging Needs for Electric Taxis')
plt.xlabel('Number of Charges Needed')
plt.ylabel('Number of Taxis')
plt.grid(True)
plt.show()

# 路网分析
-- 交叉口和道路信息分析
-- 兴趣点（POI）分析
## 交叉口和道路信息分析
结合路网数据分析交通流量路径
使用 df_node_droped 和 df_link_droped 数据框来分析交通流量路径，确定高流量的道路和交叉口。

In [ ]:
# 抽样节点和链接数据，例如抽样10%的数据
df_node_sampled = df_node_droped.sample(frac=0.1, random_state=1)
df_link_sampled = df_link_droped.sample(frac=0.1, random_state=1)

# 构建 KD 树
node_coords = df_node_sampled[['x_coord', 'y_coord']].values
node_id_to_index = {node_id: index for index, node_id in enumerate(df_node_sampled['node_id'])}
kdtree = KDTree(node_coords)

# 定义计算流量的方法
def calculate_traffic_flow(link, kdtree, node_id_to_index, node_coords):
    try:
        from_node_index = node_id_to_index[link['from_node_id']]
        to_node_index = node_id_to_index[link['to_node_id']]
        from_node = node_coords[from_node_index]
        to_node = node_coords[to_node_index]
        dist, idx = kdtree.query((from_node + to_node) / 2)
        return dist
    except KeyError:
        return np.inf

# 并行计算流量
results = Parallel(n_jobs=-1)(delayed(calculate_traffic_flow)(row, kdtree, node_id_to_index, node_coords) for _, row in df_link_sampled.iterrows())
df_link_sampled['traffic_flow'] = results

# 选择流量最高的链接
top_traffic_links = df_link_sampled.nlargest(50, 'traffic_flow')
top_traffic_coords = top_traffic_links[['from_node_id', 'to_node_id']].values.flatten()

# 可视化流量最高的链接
plt.figure(figsize=(12, 8))
plt.scatter(node_coords[:, 0], node_coords[:, 1], c='blue', marker='o', alpha=0.5, label='Nodes')
top_traffic_coords_indices = [node_id_to_index[node_id] for node_id in top_traffic_coords if node_id in node_id_to_index]
plt.scatter(node_coords[top_traffic_coords_indices, 0], node_coords[top_traffic_coords_indices, 1], c='red', marker='x', s=100, label='Top Traffic Nodes')
plt.title('Top Traffic Nodes')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 选择所有非 NaN 的 name 值，并将其标记为重要的兴趣点
df_poi_non_nan = df_poi.dropna(subset=['name']).copy()  # 使用 .copy() 创建副本

# 将几何中心转换为经纬度坐标
def parse_coordinates(centroid):
    try:
        if isinstance(centroid, float):
            return [None, None]
        coords = centroid.replace('POINT (', '').replace(')', '').split()
        return [float(coords[0]), float(coords[1])]
    except Exception as e:
        print(f"Error parsing {centroid}: {e}")
        return [None, None]

poi_coords = df_poi_non_nan['centroid'].apply(parse_coordinates)
df_poi_non_nan['longitude'] = poi_coords.apply(lambda x: x[0])
df_poi_non_nan['latitude'] = poi_coords.apply(lambda x: x[1])

# 过滤掉无效的坐标
df_poi_non_nan = df_poi_non_nan.dropna(subset=['longitude', 'latitude'])

# 可视化兴趣点分布
plt.figure(figsize=(12, 8))
plt.scatter(df_poi_non_nan['longitude'], df_poi_non_nan['latitude'], c='red', marker='o', alpha=0.5, label='Important POIs')
plt.title('Distribution of Important POIs')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# 删除之前创建的类
if 'FitnessMax' in creator.__dict__:
    del creator.FitnessMax
if 'Individual' in creator.__dict__:
    del creator.Individual

# 假设 df_taxi, df_node_droped, df_link_droped, df_poi 数据框已经被加载，包含 'longitude' 和 'latitude' 列

# 抽样数据，例如采样5%的数据
#sampled_data = df_taxi.sample(frac=0.5, random_state=1)
sampled_data = df_taxi
coords = sampled_data[['longitude', 'latitude']].values

# 使用 MiniBatchKMeans 聚类，设置 n_clusters=20
optimal_k = 21
kmeans = MiniBatchKMeans(n_clusters=optimal_k, batch_size=10000, n_init=10, random_state=1)
kmeans.fit(coords)
sampled_data['cluster'] = kmeans.labels_

# 获取聚类中心
cluster_centers = kmeans.cluster_centers_

# 使用现有的数据进行充电桩位置优化
# 抽样节点和链接数据，例如抽样5%的数据
#df_node_sampled = df_node_droped.sample(frac=0.5, random_state=1)
#df_link_sampled = df_link_droped.sample(frac=0.5, random_state=1)
df_node_sampled = df_node_droped
df_link_sampled = df_link_droped

# 构建 KDTree 用于节点坐标
node_coords = df_node_sampled[['x_coord', 'y_coord']].values
node_id_to_index = {node_id: index for index, node_id in enumerate(df_node_sampled['node_id'])}
kdtree = KDTree(node_coords)

# 重要兴趣点分析
df_poi_non_nan = df_poi.dropna(subset=['name']).copy()  # 使用 .copy() 创建副本

# 解析坐标的方法
def parse_coordinates(centroid):
    try:
        if isinstance(centroid, float):
            return [None, None]
        coords = centroid.replace('POINT (', '').replace(')', '').split()
        return [float(coords[0]), float(coords[1])]
    except Exception as e:
        print(f"Error parsing {centroid}: {e}")
        return [None, None]

# 应用解析坐标的方法
poi_coords = df_poi_non_nan['centroid'].apply(parse_coordinates)
df_poi_non_nan['longitude'] = poi_coords.apply(lambda x: x[0])
df_poi_non_nan['latitude'] = poi_coords.apply(lambda x: x[1])
df_poi_non_nan = df_poi_non_nan.dropna(subset=['longitude', 'latitude'])

# 合并所有候选位置并进行采样
combined_coords = np.vstack([cluster_centers, node_coords, df_poi_non_nan[['longitude', 'latitude']].values])
sample_size = min(500, len(combined_coords))  # 限制样本数量，确保计算不会过于耗时
all_coords = combined_coords[np.random.choice(combined_coords.shape[0], size=sample_size, replace=False)]

# 定义分块函数
def chunkify(lst, n):
    return [lst[i::n] for i in range(n)]

# 将候选位置分块
num_chunks = 5  # 减少块数以加快计算
chunks = chunkify(all_coords, num_chunks)

# 定义遗传算法参数
POPULATION_SIZE = 100  # 减少种群规模以加快计算
GENERATIONS = 50  # 减少代数以加快计算
MUTATION_RATE = 0.02
CROSSOVER_RATE = 0.8
COVERAGE_RADIUS = 3.0  # 充电桩覆盖半径3公里

# 定义适应度函数
def evaluate(individual, coords_part, all_coords):
    selected_coords = [coords_part[i] for i in range(len(individual)) if individual[i] == 1]
    total_coverage = 0
    for point in all_coords:
        covered = any(np.linalg.norm(coord - point) <= COVERAGE_RADIUS for coord in selected_coords)
        total_coverage += covered
    # 惩罚条件：每个未覆盖的点增加一个大的惩罚
    if total_coverage < len(all_coords):
        return total_coverage - (len(all_coords) - total_coverage) * 100,
    return total_coverage,

# 初始化遗传算法工具箱
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(chunks[0]))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# 定义分块求解函数
def solve_chunk(chunk, all_coords):
    toolbox.register("evaluate", evaluate, coords_part=chunk, all_coords=all_coords)
    population = toolbox.population(n=POPULATION_SIZE)
    result, logbook = algorithms.eaSimple(population, toolbox, cxpb=CROSSOVER_RATE, mutpb=MUTATION_RATE,
                                          ngen=GENERATIONS, verbose=False)
    best_individual = tools.selBest(population, k=1)[0]
    return [chunk[i] for i in range(len(best_individual)) if best_individual[i] == 1]

# 分块求解
charging_station_locations = []
for chunk in tqdm(chunks, desc="Solving Chunks"):
    charging_station_locations.extend(solve_chunk(chunk, all_coords))

In [ ]:
# 可视化充电桩位置
print(len(charging_station_locations))
plt.figure(figsize=(12, 8))
#plt.scatter(df_taxi['longitude'], df_taxi['latitude'], c='blue', marker='o', alpha=0.1, label='Taxi Trajectories')
plt.scatter([loc[0] for loc in charging_station_locations], [loc[1] for loc in charging_station_locations], c='green', marker='x', s=100, label='Charging Stations')
plt.title('Optimal Charging Station Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 创建北京地图
beijing_map = folium.Map(location=[39.9042, 116.4074], zoom_start=11)

# 添加出租车轨迹点（仅采样一部分以加快绘制速度）
#taxi_sample = df_taxi.sample(frac=0.01, random_state=1)
#taxi_locations = taxi_sample[['latitude', 'longitude']].values.tolist()
#FastMarkerCluster(taxi_locations).add_to(beijing_map)

# 添加充电桩位置
for loc in charging_station_locations:
    folium.Marker(location=[loc[1], loc[0]], popup='Charging Station', icon=folium.Icon(color='green')).add_to(beijing_map)

# 显示地图
beijing_map.save('beijing_charging_stations.html')
beijing_map

## 分析acn数据

In [ ]:
df_acn = pd.read_csv('/kaggle/input/mcm-jnu-a-problem/datasets/acn.csv')

print(df_acn.head())

# 重命名列
df_acn.columns = ['插入充电桩时间', '拔出充电桩时间', '星期', '周末', '到达停车位时间', '充电持续时间（min）', '保持连接但不充电时间（min）', '停留时间（min）', '充电量（kWh）', '地点']

# 转换时间列为日期时间格式
df_acn['插入充电桩时间'] = pd.to_datetime(df_acn['插入充电桩时间'])
df_acn['拔出充电桩时间'] = pd.to_datetime(df_acn['拔出充电桩时间'])
df_acn['到达停车位时间'] = pd.to_datetime(df_acn['到达停车位时间'])

# 提取日期和时间信息
df_acn['year'] = df_acn['插入充电桩时间'].dt.year
df_acn['month'] = df_acn['插入充电桩时间'].dt.month
df_acn['day'] = df_acn['插入充电桩时间'].dt.day
df_acn['hour'] = df_acn['插入充电桩时间'].dt.hour
df_acn['weekday'] = df_acn['插入充电桩时间'].dt.weekday

# 按日期聚合数据，计算每天的充电需求
daily_demand_acn = df_acn.groupby(df_acn['插入充电桩时间'].dt.date).agg({
    '充电量（kWh）': 'sum',
    '充电持续时间（min）': 'sum',
    '保持连接但不充电时间（min）': 'sum',
    '停留时间（min）': 'sum'
}).reset_index()

# 重命名日期列
daily_demand_acn.rename(columns={'插入充电桩时间': '日期'}, inplace=True)

print(daily_demand_acn.head())

## 研究北京市朝阳区一块选定区域

In [ ]:
import networkx as nx
#新增加的两行
import matplotlib
matplotlib.rc("font",family='YouYuan')

# 节点信息：路口编号、位置（x, y 坐标）、功能区域
nodes = [
    (1, {'pos': (116.456941,39.919094), 'type': '生活区'}),
    (2, {'pos': (116.464231,39.919042), 'type': '生活区'}),
    (3, {'pos': (116.456937,39.91788), 'type': '工作区'}),
    (4, {'pos': (116.459398,39.917811), 'type': '工作区'}),
    (5, {'pos': (116.46247,39.917451), 'type': '生活区'}),
    (6, {'pos': (116.464509,39.9175), 'type': '商业区'}),
    (7, {'pos': (116.456892,39.915168), 'type': '工作区'}),
    (8, {'pos': (116.459416,39.915404), 'type': '工作区'}),
    (9, {'pos': (116.462452,39.915092), 'type': '生活区'}),
    (10, {'pos': (116.456865,39.914338), 'type': '商业区'}),
    (11, {'pos': (116.462452,39.914207), 'type': '商业区'}),
    (12, {'pos': (116.464482,39.914152), 'type': '商业区'}),
    (13, {'pos': (116.458183,39.914186), 'type': '生活区'}),
    (14, {'pos': (116.456889,39.911481), 'type': '生活区'}),
    (15, {'pos': (116.458183,39.911454), 'type': '生活区'}),
    (16, {'pos': (116.460413,39.911457), 'type': '工作区'}),
    (17, {'pos': (116.462443,39.912308), 'type': '工作区'}),
    (18, {'pos': (116.464365,39.912315), 'type': '商业区'}),
    (19, {'pos': (116.456847,39.909589), 'type': '工作区'}),
    (20, {'pos': (116.458311,39.909721), 'type': '商业区'}),
    (21, {'pos': (116.462389,39.909949), 'type': '商业区'}),
    (22, {'pos': (116.464357,39.909894), 'type': '商业区'}),
    (23, {'pos': (116.468462,39.919156), 'type': '工作区'}),
    (24, {'pos': (116.471929,39.919163), 'type': '商业区'}),
    (25, {'pos': (116.473735,39.919219), 'type': '工作区'}),
    (26, {'pos': (116.46848,39.917344), 'type': '商业区'}),
    (27, {'pos': (116.471873,39.917399), 'type': '商业区'}),
    (28, {'pos': (116.473816,39.917413), 'type': '商业区'}),
    (29, {'pos': (116.468345,39.914096), 'type': '商业区'}),
    (30, {'pos': (116.474049,39.914082), 'type': '商业区'}),
    (31, {'pos': (116.476187,39.919264), 'type': '工作区'}),
    (32, {'pos': (116.484523,39.919347), 'type': '工作区'}),
    (33, {'pos': (116.479924,39.917437), 'type': '工作区'}),
    (34, {'pos': (116.484416,39.917458), 'type': '工作区'}),
    (35, {'pos': (116.479915,39.91411), 'type': '商业区'}),
    (36, {'pos': (116.484434,39.914138), 'type': '生活区'}),
    (37, {'pos': (116.468325,39.912266), 'type': '工作区'}),
    (38, {'pos': (116.468316,39.910136), 'type': '工作区'}),
    (39, {'pos': (116.472403,39.913954), 'type': '工作区'}),
    (41, {'pos': (116.472358,39.912177), 'type': '工作区'}),
    (42, {'pos': (116.471235,39.911623), 'type': '工作区'}),
    (43, {'pos': (116.471208,39.910246), 'type': '工作区'}),
    (44, {'pos': (116.47384,39.910447), 'type': '生活区'}),
    (45, {'pos': (116.476122,39.913906), 'type': '工作区'}),
    (46, {'pos': (116.478898,39.913941), 'type': '商业区'}),
    (47, {'pos': (116.480757,39.913989), 'type': '商业区'}),
    (49, {'pos': (116.475871,39.912813), 'type': '工作区'}),
    (50, {'pos': (116.478907,39.912896), 'type': '工作区'}),
    (51, {'pos': (116.478925,39.91154), 'type': '生活区'}),
    (52, {'pos': (116.480703,39.911637), 'type': '商业区'}),
    (53, {'pos': (116.476023,39.910689), 'type': '工作区'}),
    (54, {'pos': (116.478907,39.910828), 'type': '生活区'}),
    (55, {'pos': (116.480757,39.910848), 'type': '生活区'}),
    (56, {'pos': (116.484342,39.911007), 'type': '商业区'})

    
]

# 边信息：起点路口编号、终点路口编号
edges = [
    (1, 2),
    (1, 3),
    (2, 6),
    (3, 4),
    (3, 7),
    (4, 5),
    (4, 8),
    (5, 6),
    (5, 9),
    (6, 12),
    (7, 8),
    (7, 10),
    (8, 9),
    (9, 11),
    (10, 13),
    (10, 14),
    (11, 12),
    (13, 15),
    (13, 11),
    (12, 18),
    (14, 15),
    (14, 19),
    (15, 16),
    (15, 20),
    (16, 17),
    (17, 18),
    (17, 21),
    (18, 22),
    (19, 20),
    (20, 21),
    (21, 22),
    (2, 23),
    (23, 24),
    (24, 25),
    (24, 27),
    (25, 28),
    (6, 26),
    (26, 27),
    (27, 28),
    (12, 29),
    (28, 30),
    (25, 31),
    (31, 32),
    (28, 33),
    (33, 34),
    (33, 35),
    (34, 36),
    (32, 34),
    (29, 39),
    (39, 30),
    (18, 37),
    (22, 38),
    (29, 37),
    (37, 38),
    (38, 43),
    (43, 44),
    (41, 42),
    (42, 43),
    (30, 44),
    (37, 41),
    (39, 41),
    (45, 46),
    (45, 49),
    (46, 47),
    (46, 50),
    (47, 36),
    (47, 52),
    (36, 56),
    (49 ,50),
    (49, 53),
    (50, 51),
    (51, 52),
    (51, 54),
    (52, 55),
    (53, 54),
    (30, 45),
    (44, 53),
    (54, 55),
    (55, 56)
     

]
 
 
 

# 创建图
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# 定义节点颜色
color_map = {
    '商业区': 'pink',
    '生活区': 'yellow',
    '工作区': 'skyblue'
}

# 获取节点位置
pos = nx.get_node_attributes(G, 'pos')

# 获取节点颜色
node_colors = [color_map[G.nodes[node]['type']] for node in G.nodes]

# 绘制网络图
plt.figure(figsize=(6, 6))
nx.draw(G, pos, node_color=node_colors, with_labels=True, node_size=200, font_size=7, font_color='black', font_weight='bold')

# 添加图例
legend_labels = {'pink': '商业区', 'yellow': '生活区', 'skyblue': '工作区'}
for color, label in legend_labels.items():
    plt.plot([], [], marker='o', color=color, linestyle='', markersize=10, label=label)
plt.legend()

# 设置标题
plt.title('分割后研究区域')
plt.savefig('beijing_network.png')
plt.show()


In [ ]:
# 将datetime列转换为datetime类型
df_taxi['datetime'] = pd.to_datetime(df_taxi['datetime'])

# 按出租车ID和时间排序
df_taxi = df_taxi.sort_values(by=['taxi_id', 'datetime'])

# 计算时间差和距离差
df_taxi['time_diff'] = df_taxi.groupby('taxi_id')['datetime'].diff().dt.total_seconds() / 60
df_taxi['distance'] = ((df_taxi['longitude'] - df_taxi.groupby('taxi_id')['longitude'].shift())**2 + 
                       (df_taxi['latitude'] - df_taxi.groupby('taxi_id')['latitude'].shift())**2)**0.5

# 判断是否有充电需求（时间差大于10分钟且距离差为0）
df_taxi['charging_need'] = (df_taxi['time_diff'] > 10) & (df_taxi['distance'] == 0)

# 筛选有充电需求的数据
charging_needs = df_taxi[df_taxi['charging_need']].copy()

# 提取日期和小时
charging_needs['date'] = charging_needs['datetime'].dt.date
charging_needs['hour'] = charging_needs['datetime'].dt.hour

# 按日期和小时统计充电需求
charging_demand = charging_needs.groupby(['date', 'hour']).size().reset_index(name='demand')

# 创建时间索引
charging_demand['datetime'] = pd.to_datetime(charging_demand['date']) + pd.to_timedelta(charging_demand['hour'], unit='h')
charging_demand.set_index('datetime', inplace=True)

# 按小时重新采样数据
hourly_demand = charging_demand['demand'].resample('h').sum().fillna(0)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 拟合SARIMA模型
model = SARIMAX(hourly_demand, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
results = model.fit()

# 预测未来的充电需求
future_steps = 24 * 365 * 5  # 未来5年
forecast = results.get_forecast(steps=future_steps)
forecast_index = pd.date_range(start=hourly_demand.index[-1], periods=future_steps + 1, freq='h')[1:]
forecast_df = pd.DataFrame({'forecast': forecast.predicted_mean}, index=forecast_index)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 准备数据用于3D绘图
hourly_demand = hourly_demand.reset_index()
hourly_demand['day_of_year'] = hourly_demand['datetime'].dt.dayofyear
hourly_demand['hour_of_day'] = hourly_demand['datetime'].dt.hour

forecast_df = forecast_df.reset_index()
forecast_df.columns = ['datetime', 'forecast']
forecast_df['day_of_year'] = forecast_df['datetime'].dt.dayofyear
forecast_df['hour_of_day'] = forecast_df['datetime'].dt.hour

# 创建3D图
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制历史数据
ax.scatter(hourly_demand['day_of_year'], hourly_demand['hour_of_day'], hourly_demand['demand'], c='blue', label='Observed Demand', alpha=0.6)

# 绘制预测数据
ax.scatter(forecast_df['day_of_year'], forecast_df['hour_of_day'], forecast_df['forecast'], c='red', label='Forecasted Demand', alpha=0.6)

# 设置轴标签
ax.set_xlabel('Day of Year')
ax.set_ylabel('Hour of Day')
ax.set_zlabel('Charging Demand')

# 设置标题
ax.set_title('3D Plot of EV Charging Demand in Beijing (2021-2025)')
ax.legend()

plt.show()
